Let's see the performance of the numerical integrator for solving the following stiff, linear ODE system. This file solves the following ordinary differential equation (ODE) system:

\begin{eqnarray*}
	\frac{dy_1(t)}{dt} &=& \lambda_1 ( y_1(t) -  \cos(t) ) - \sin(t) \\
	\frac{dy_2(t)}{dt} &=& \lambda_2 ( y_2(t) -  \cos(t) ) - \sin(t) \\
	\frac{dy_3(t)}{dt} &=& \lambda_3 ( y_3(t) -  \cos(t) ) - \sin(t)
\end{eqnarray*}

where the initial condition is $y_1(0) = y_2(0) = y_3(0) = 1$. $\lambda_1, \lambda_2, \lambda_3$ are constants where $\lambda_{i} \le 0$ and control the stiffness of 
the sytem.

The exact solution for this system is 
\begin{equation*}
	y_1(t) = y_2(t)= y_3(t) = \cos(t)
\end{equation*}

Import

In [1]:
import sys
sys.path.append('..//source')

import matplotlib.pylab as plt
import numpy as np
import numpy.linalg as LA

from scipy.interpolate import CubicSpline
import scipy, time 

import analysis, integrator, params, plotter

import points

In [2]:
%matplotlib notebook

--- 
Functions

---

In [3]:
# the stiffness parameter
K_stiff = -1/np.pi * np.array( (1e-3, 1e2, 1e5) )


def exact(t, m):
    
    n = len(t)
    y = np.zeros( (n, m) )
    
    for i in range(m):
        y[:,i] = np.cos(t)
    
    return y

def f_eval(t, y):
    
    """
    sovling
    dy_i/dt = k_i * (y_i - cos(t) ) - sin(t)
    
    :param float t: the time node
    :param numpy.ndarray: y the approximate solution length( m) at time node t
    """

    F = np.zeros( y.shape )
    m = len(y)
    
    # get the component-wise interaction
    for i in range(m):              
        F[i] = K_stiff[i] * ( y[i] - np.cos(t) ) - np.sin(t)
        
    return F

def get_corrections_norm(D):
    
    """
    :param numpy.ndarray D: dimensions (the number of iterations, the number of nodes, the dimension of the problem)
    """
    
    y = np.vstack( [ np.linalg.norm(x, axis=0) for x in D] )
    
    return y

def get_stuff(p, dt):
        
    if p.t[-1] != 1:
        t = dt * np.hstack( [p.t, 1])
    else:
        t = dt * p.t
    
    S, S_p = dt * p.S, dt * p.S_p
    
    spectral_radius = p.spectral_radius
    node_type = p.node_type
    
    return t, S, S_p, spectral_radius, node_type


---
Parameters

---

In [4]:
#
# set up integration parameters
#

# the parameters of the simulation
n_nodes     = 5

# the time step
#dt          = np.pi / 4
dt          = 1.0

# the dimension of the problem
m           = 3

# the initial solution
y0          = np.ones( (m,) )

# create the parameter object for Gauss-Lobatto nodes
p_lobatto = params.Params(n_nodes=n_nodes, m=m, node_type=points.GAUSS_LOBATTO)

# create paramter for radau
#p_radau  = params.Params(n_nodes=n_nodes, m=m, node_type=points.GAUSS_RADAU)

# create paramter for radau 2a
#p_radau_2a = params.Params(n_nodes=n_nodes, m=m, node_type=points.GAUSS_RADAU_2A)

# legendre nodes
#p_legendre = params.Params(n_nodes=n_nodes, m=m, node_type=points.GAUSS_LEGENDRE)

# backward euler, implicit tolerance. Also sused in SDC
BE_TOL = 1e-12

# SDC solver tolerance
SDC_TOL = 1e-12

# the maximum number of Newton iterations in the JFNK solver
N_ITER_MAX_NEWTON=integrator.N_ITER_MAX_NEWTON

---
Run one time step

---

Exact solution

In [5]:
y_exact = exact(dt * p_lobatto.t, m)

Backward Euler

In [6]:
nodes, S, S_p, spectral_radius, node_type = get_stuff(p_lobatto, dt)
y_be = integrator.backward_euler(f_eval, nodes, y0, S_p, be_tol=BE_TOL, do_print=True)

t = nodes
LA.norm( y_be - y_exact, ord=np.inf)

elapsed time: 0.00[s]


0.11507996569196383

SDC

In [7]:
t, S, S_p, spectral_radius, node_type = get_stuff(p_lobatto, dt)

# get the provisional solution
y_be = integrator.backward_euler(f_eval, t, y0, S_p, be_tol=BE_TOL)

# run SDC
y_sdc, Y_sdc, D_sdc, is_converged_sdc = integrator.sdc(f_eval, t, y0, y_be, S, S_p, be_tol=BE_TOL, sdc_tol=SDC_TOL, do_print=True)
print('SDC converged: %s ' % is_converged_sdc)

# the norm of the corrections
d_norm_sdc, log_d_norm_sdc, d_norm_rel_sdc, log_d_norm_rel_sdc = analysis.analyze_corrections(D_sdc, Y_sdc)

elapsed time: 0.09[s]
SDC converged: True 


..//source/analysis.py:140: RuntimeWarning: divide by zero encountered in log10
  log_d_norm      = np.log10(d_norm)
..//source/analysis.py:141: RuntimeWarning: divide by zero encountered in log10
  log_d_norm_rel  = np.log10(d_norm_rel)


JFNK

In [8]:
t, S, S_p, spectral_radius, node_type = get_stuff(p_lobatto, dt)

# get the provisional solution
y_be = integrator.backward_euler(f_eval, t, y0, S_p, be_tol=BE_TOL)

# run the JFNK
y_jfnk, Y_jfnk, D_jfnk, is_converged_jfnk, is_stiff,ratios = integrator.jfnk(f_eval, t, y0, y_be, S, S_p, spectral_radius, be_tol=BE_TOL, \
                                                                          sdc_tol=SDC_TOL, do_print=True)
print('JFNK converged: %s ' % is_converged_jfnk)

# the norm of the corrections
d_norm_jfnk, log_d_norm_jfnk, d_norm_rel_jfnk, log_d_norm_rel_jfnk = analysis.analyze_corrections(D_jfnk, Y_jfnk)

elapsed time: 0.02[s]
JFNK converged: True 


..//source/integrator.py:721: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, res, rank, s = np.linalg.lstsq(B, rhs)
..//source/analysis.py:140: RuntimeWarning: divide by zero encountered in log10
  log_d_norm      = np.log10(d_norm)
..//source/analysis.py:141: RuntimeWarning: divide by zero encountered in log10
  log_d_norm_rel  = np.log10(d_norm_rel)


Spectral solution

In [9]:
t, S, S_p, spectral_radius, node_type = get_stuff(p_lobatto, dt)

# provisional solution
y_be = integrator.backward_euler(f_eval, t, y0, S_p, be_tol=BE_TOL)

# spectral solution
y_spect = integrator.run_spectral(f_eval, t, y0, S, tol=1e-10, y_approx=y_be, verbose=True)

0:  |F(x)| = 1.67226e-05; step 1; tol 3.29637e-09
1:  |F(x)| = 2.44356e-10; step 1; tol 1.92168e-10
2:  |F(x)| = 1.38914e-12; step 1; tol 2.90862e-05


Plot the magnitude of deferred corrections (both SDC and JFNK corrections)

In [10]:
plt.figure()
plt.title('Magnitude of corrections ')

plt.plot( range(log_d_norm_sdc.shape[0]), log_d_norm_sdc.max(axis=1), '-o', label='sdc')
plt.plot( range(log_d_norm_jfnk.shape[0]), log_d_norm_jfnk.max(axis=1), '-o', label='jfnk')

plt.xlabel('SDC iteration')
plt.ylabel('log10 (error)')
plt.legend(loc='best')

plt.show()

---
Run multiple time steps

---

In [12]:
# final time
t_final = np.sqrt(4/3) * np.pi

# absolute error tolerance for tol
tol = 1e-11

# initial time step for the adaptive JFNK algorithm
dt_init = t_final / 10

print(t_final, dt_init)

3.6275987284684352 0.3627598728468435


**Adaptive** JFNK

In [12]:
t_adapt, y_adapt, Y_adapt, D_adapt, h_adapt = integrator.jfnk_adaptive(f_eval, t_init=0, t_final=t_final, dt_init=dt_init, p=p_lobatto, y0=y0, \
                                                                       tol=tol, do_print=True)

..//source/integrator.py:721: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, res, rank, s = np.linalg.lstsq(B, rhs)


elapsed time: 1.94[s]


**Uniform** JFNK

In [13]:
t_uni, y_uni, Y_uni, D_uni = integrator.jfnk_uniform(f_eval, t_init=0, t_final=t_final, n_steps=12, p=p_lobatto, y0=y0, do_print=True)

..//source/integrator.py:721: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, res, rank, s = np.linalg.lstsq(B, rhs)


elapsed time: 0.27[s]


**Reference** solution

In [14]:
# the reference solution
n_steps_ref = 100
p_ref = params.Params(n_nodes=10, m=m, node_type=points.GAUSS_LOBATTO)

t_ref, y_ref, Y_ref, D_ref = integrator.jfnk_uniform(f_eval, t_init=0, t_final=t_final, n_steps=n_steps_ref, p=p_ref, y0=y0, do_print=True)

# spline
y_spline = CubicSpline(t_ref, y_ref)

# errors
f = lambda t, y: ( LA.norm( y_spline(t) - y, ord=np.inf), integrator.relative_norm(y_spline(t) - y, y_spline(t)) )

..//source/integrator.py:721: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c, res, rank, s = np.linalg.lstsq(B, rhs)


elapsed time: 10.46[s]


Comparing the absolute and relative erros of the **uniform** solution to the reference solution

In [15]:
f(t_uni, y_uni)

(3.5825586941484744e-09, 1.5841824829072928e-09)

Comparing the absolute and relative errors of the **adaptive** solution to the reference solution

In [16]:
f(t_adapt, y_adapt)

(2.132772847218689e-10, 3.411275342907868e-11)

---
Plot the JFNK corrections for each time step 

---

In [17]:
plotter.plot_correction_time_steps(D_uni, Y_uni)

plt.show()

Plot norm of corrections comparing 

In [18]:
y_spect_norm = LA.norm(y_spect, axis=0)

labels=['y1', 'y2', 'y3']

plotter.corrections(d_norm_sdc, d_norm_jfnk, y_spect_norm, do_rerr=True, labels=labels, do_legend=True)
plt.show()

..//source/plotter.py:150: RuntimeWarning: divide by zero encountered in log10
  temp = ax.plot(np.log10(y), ls, label=label)
/opt/local/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/legend.py:337: UserWarning: Automatic legend placement (loc="best") not implemented for figure legend. Falling back on "upper right".
  warnings.warn('Automatic legend placement (loc="best") not '


---
Errors

---

calculate absolute errors, relative errors

In [20]:
# absolute error, relative error

# error exact solution vs. spectral solution
aerr_spect, rerr_spect = analysis.error_analysis(y_spect, y_exact)

# spectral solution vs. backward euler solution
aerr_be, rerr_be = analysis.error_analysis(y_be, y_spect)

# spectral solution vs. SDC solution
aerr_sdc, rerr_sdc = analysis.error_analysis(y_sdc, y_spect)

# spectral solution vs. JFNK solution
aerr_jfnk, rerr_jfnk = analysis.error_analysis(y_jfnk, y_spect)